In [3]:
import numpy as np

class DecisionStumpRegressor:
    """
    Simple decision stump regressor (1-level decision tree).
    """
    def __init__(self):
        self.feature_index = None
        self.threshold = None
        self.left_value = None
        self.right_value = None

    def fit(self, X, y):
        m, n = X.shape
        min_error = float("inf")
        for feature in range(n):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask
                if np.any(left_mask) and np.any(right_mask):
                    left_value = np.mean(y[left_mask])
                    right_value = np.mean(y[right_mask])
                    y_pred = np.where(left_mask, left_value, right_value)
                    error = np.mean((y - y_pred) ** 2)
                    if error < min_error:
                        min_error = error
                        self.feature_index = feature
                        self.threshold = threshold
                        self.left_value = left_value
                        self.right_value = right_value

    def predict(self, X):
        mask = X[:, self.feature_index] <= self.threshold
        return np.where(mask, self.left_value, self.right_value)

class GradientBoostingRegressor:
    """
    Gradient Boosting Regressor with decision stumps as weak learners.
    """
    def __init__(self, n_estimators=100, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.trees = []
        self.gammas = []

    def fit(self, X, y):
        # Initial prediction is the mean of y
        y_pred = np.full(y.shape, np.mean(y))
        self.init_ = np.mean(y)
        for _ in range(self.n_estimators):
            residuals = y - y_pred
            tree = DecisionStumpRegressor()
            tree.fit(X, residuals)
            update = tree.predict(X)
            y_pred += self.learning_rate * update
            self.trees.append(tree)
            self.gammas.append(self.learning_rate)

    def predict(self, X):
        y_pred = np.full((X.shape[0],), self.init_)
        for tree, gamma in zip(self.trees, self.gammas):
            y_pred += gamma * tree.predict(X)
        return y_pred

> ## Example usage:

In [4]:
# Simple regression dataset
X = np.array([[1], [2], [3], [4], [5], [6]])
y = np.array([1.2, 1.9, 3.0, 3.9, 5.1, 6.2])

gb = GradientBoostingRegressor(n_estimators=10, learning_rate=0.1)
gb.fit(X, y)
preds = gb.predict(np.array([[1.5], [3.5], [5.5]]))
print("Gradient Boosting predictions:", preds)

Gradient Boosting predictions: [2.4904795  3.82551431 4.71066587]
